In [22]:
import torch
#from denoising_diffusion_pytorch import Unet, GaussianDiffusion
#from denoising_diffusion_pytorch import Unet1D, GaussianDiffusion1D
from denoising_diffusion_pytorch import *
import numpy as np
import matplotlib.pyplot as plt
#from scipy.stats import skew,kurtosis
from PIL import Image

In [3]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


In [ ]:
pip install torch torchvision numpy matplotlib einops ema_pytorch accelerate pytorch_fid

## Load the data

In [12]:
num_testsub = 32
num_classes = 4
num_channel = 40
num_datapoints = 8064
num_trials = 40
sampling_rate = 128 # 128Hz as given in the data

def loadfiles_normalized():
    data_dict = {}
    print("Loading files into data_dict .................")
    for i in range(num_testsub):
                    if i < 10:
                        name = '%0*d' % (2,i+1)
                    else:
                        name = i+1
                    fname = '../persistent/data/noramlized_datasub'+str(name) +'.npy'
                    data_dict["sub%s" %name] = np.load(fname)    
    print("Loaded!!!!!") 
    return data_dict

In [13]:
data = loadfiles_normalized()

Loading files into data_dict .................
Loaded!!!!!


In [14]:
# (40, 40, 99) (videonumber/trialnumber , channelnumber, datapoints)
data['sub01'].shape

(40, 40, 99)

In [15]:
# matrix 1
data['sub01'][0].shape

(40, 99)

In [24]:
matrix = np.zeros((880,40,104))
matrix[:] = np.nan
counter = 0
for sub in data.keys():
    if sub == 'sub23':
        break
    for i in range(data[sub][0].shape[0]):
        temp = data[sub][i]
#        print(temp.shape)
#        temp = temp[: , :96]
        # pad the array with a column of zeros
        temp = np.pad(temp, ((0, 0), (0, 5)), mode='constant')
        matrix[counter, :] = temp
        counter = counter + 1

In [25]:
matrix.shape

(1280, 40, 104)

In [20]:
!mkdir images2
for i in range(880):
    #transpose = np.transpose(matrix[i,:,:], (1,2,0))
    tra = np.multiply(matrix[i,:,:], 255).astype(np.uint8)
    img = Image.fromarray(tra)
    img.save("images2/train"+str(i)+".png")

mkdir: cannot create directory ‘images2’: File exists


----

DEPCREATED

In [ ]:
matrix = np.zeros((1280,3,40,104))
matrix[:] = np.nan
counter = 0
check = np.zeros((104,40))
for sub in data.keys():
    for i in range(data[sub][0].shape[0]):
        temp = data[sub][i]
#        print(temp.shape)
#        temp = temp[: , :96]
        # pad the array with a column of zeros
        temp = np.pad(temp, ((0, 0), (0, 5)), mode='constant')
        check = temp
        matrix[counter, :] = temp
        counter = counter + 1

In [26]:
matrix = np.zeros((1280,1,40,104))
matrix[:] = np.nan
counter = 0
check = np.zeros((104,40))
for sub in data.keys():
    for i in range(data[sub][0].shape[0]):
        temp = data[sub][i]
#        print(temp.shape)
#        temp = temp[: , :96]
        # pad the array with a column of zeros
        temp = np.pad(temp, ((0, 0), (0, 5)), mode='constant')
        check = temp
        matrix[counter, :] = temp
        counter = counter + 1

In [27]:
matrix.shape

(1280, 1, 40, 104)

In [6]:
np.isnan(matrix).any()

False

In [ ]:
matrix[0][0]

In [28]:
tensor_matrix = torch.tensor(matrix).float()

In [8]:
tensor_matrix.shape

torch.Size([1280, 1, 40, 104])

In [ ]:
image_size = (99,40)
image_size[0]

----

In [29]:
model = Unet(
    dim = 64,
    dim_mults = (1, 2, 4, 8),
    channels = 1
)

diffusion = GaussianDiffusion(
    model,
    image_size = (40,104),
    timesteps = 1000,   # number of steps
    loss_type = 'l1'    # L1 or L2
)

training_images = tensor_matrix # images are normalized from 0 to 1
#training_images = torch.rand(8, 3, 96, 40) # images are normalized from 0 to 1
#loss = diffusion(training_images)
#loss.backward()

# after a lot of training

#sampled_images = diffusion.sample(batch_size = 4)
#sampled_images.shape # (4, 3, 128, 128)

In [ ]:
for i in range(1000):
    loss = diffusion(training_images)
    loss.backward()

In [40]:
model = Unet(
    dim = 64,
    dim_mults = (1, 2, 4, 8),
    channels = 1
)

diffusion = GaussianDiffusion(
    model,
    image_size = (40,104),
    timesteps = 1000,           # number of steps
    sampling_timesteps = 250,   # number of sampling timesteps (using ddim for faster inference [see citation for ddim paper])
    loss_type = 'l2'            # L1 or L2
)

trainer = Trainer(
    diffusion,
    'images2/',
    train_batch_size = 32,
    train_lr = 8e-5,
    train_num_steps = 2000,         # total training steps
    gradient_accumulate_every = 2,    # gradient accumulation steps
    ema_decay = 0.995,                # exponential moving average decay
    amp = True,                       # turn on mixed precision
    calculate_fid = True              # whether to calculate fid during training
)
#trainer.load('model-1.pt')
#trainer.train()

/usr/local/lib/python3.8/dist-packages/torch/amp/autocast_mode.py:204: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [41]:
trainer.load(1)

loading from version 1.5.6


In [42]:
trainer.train()

 50%|#####     | 1000/2000 [00:00<?, ?it/s]

sampling loop time step:   0%|          | 0/250 [00:00<?, ?it/s]

fid_score: 0.2621458597442292
training complete


In [43]:
for i in range(10):
    sampled_images = diffusion.sample(batch_size = 128)
    torch.save(sampled_images,'sampled1D_2000_128_'+str(i))
    print("Finished Sample: ",i)

sampling loop time step:   0%|          | 0/250 [00:00<?, ?it/s]

Finished Sample:  0


sampling loop time step:   0%|          | 0/250 [00:00<?, ?it/s]

Finished Sample:  1


sampling loop time step:   0%|          | 0/250 [00:00<?, ?it/s]

...

Finished Sample:  7


sampling loop time step:   0%|          | 0/250 [00:00<?, ?it/s]

Finished Sample:  8


sampling loop time step:   0%|          | 0/250 [00:00<?, ?it/s]

Finished Sample:  9
